# 実践データ科学入門 2020年度木曜4限

# 第4回 その2 正則化

In [ ]:
%matplotlib inline
#%matplotlib notebook # if necessary to rotate figures in 3D plot
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import art3d
from ipywidgets import interact
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

## 正則化とは

多項式回帰において，多項式の次数を十分上げると学習誤差を限りなく下げられることを見た．しかしこの場合は汎化誤差が大きくなる過学習を引き起こす可能性があるのであった．過学習を抑えるためには汎化誤差が上がらないような次数をテスト用のデータを用いて検証すれば良いが，学習時に過学習させないように工夫することも重要である．その一つが__正則化__と呼ばれる操作である．

最小二乗法による学習では

$$
E(a_0, a_1, \ldots, a_N) := \frac1N \sum_{i=1}^N (e^{(i)})^2 = \frac1N \sum_{i=1}^N \left( y^{(i)} - a_0 - \sum_{j=1}^M a_j x_j^{(i)} \right)^2
$$

という平均二乗誤差を最小にするパラメータ $a_0, a_1, \ldots, a_M$ を求めたのであった．

正則化最小二乗法とは，ハイパーパラメータ $\eta>0$ に対して

$$
\tilde E(a_0, a_1, \ldots, a_N; \eta) := \frac1N \sum_{i=1}^N (e^{(i)})^2 + \eta \sum_{j=0}^M (a_j)^2
= \frac1N \sum_{i=1}^N \left( y^{(i)} - a_0 - \sum_{j=1}^M a_j x_j^{(i)} \right)^2 + \eta \sum_{j=0}^M (a_j)^2
\qquad ...(*)
$$

を最小にする $a_0, a_1, \ldots, a_M$ を求める手法である．$(*)$ の右辺第二項を正則化項と呼ぶ．

一般に過学習が行われるときは，過剰な回帰変数 $x_j$ にまで非零のパラメータ $a_j$ が与えられてしまい，結果的に

$$
\sum_{j=0}^M (a_j)^2
$$

の値が大きくなっている．誤差を最小化させたくても $\sum_{j=0}^M (a_j)^2$ が大きくなってしまうなら，それは最小化問題としては罰則として働いてしまう．そのため，$(*)$ の右辺第二項のことを罰則項とも呼ぶ．

In [ ]:
# 真のパラメータ
A0 = 1.2
A1 = 5.6
A2 = 0.0

# training data
X1 = np.arange(-2, 1, 0.1)
Y1 = A0 + A1*X1 + A2*X1**2 + 2.0*np.random.randn(X1.size)

# test data
X2 = np.arange(-2, 1, 0.1)
Y2 = A0 + A1*X2 + A2*X2**2 + 2.0*np.random.randn(X2.size)

In [ ]:
# 回帰直線のプロット
def plot_XY_and_regressionline(a0=0.0, a1=1.0):
    fig = plt.figure()
    ax = plt.axes()
    ax.set_xlabel("X", size=20)
    ax.set_ylabel("Y", size=20)
    ax.set_xticks
    #ax.set_ylim(-3, 3)
    ax.scatter(X1, Y1)
    ax.plot([np.min(X1), np.max(X1)], [a0+a1*np.min(X1), a0+a1*np.max(X1)], linewidth=3, color='tab:red')
    ax.set_title('MSE = %f'%(np.sum((Y1-a0-a1*X1)**2)), size=20)
    ax.tick_params(labelsize=12)

In [ ]:
interact(plot_XY_and_regressionline, a0=(-5.0, 5.0, 0.1), a1=(-5.0, 10.0, 0.1))

# 青点がデータ点
# 赤が回帰直線
# MSE = Mean Squared Error = 平均二乗誤差

# 必ずしも真のパラメータのときに平均二乗誤差最小になるわけではないことに注意しよう

線形モデルに対して強めのノイズを付加すると多項式フィッティングを用いたくなってしまう．この場合どうなるかみてみよう．

### 多項式フィッティング（復習）

回帰変数2つの重回帰では，データ空間 $(x_1, x_2, y)$ の3次元空間に回帰平面を引き，データから平面までの高さの二乗和が最小になるように選ぶ．

In [ ]:
# フィッティングに用いる多項式の最大次数を１から1つずつ増やしていく
maxM = 20

# dataset
Xtrain = np.zeros((X1.size, 0))
Xtest = np.zeros((X2.size, 0))

# 回帰モデルの設定
reg = linear_model.LinearRegression()

MSE1 = np.zeros(maxM+1)
MSE2 = np.zeros(maxM+1)
anorm2 = np.zeros(maxM+1)
coeff = np.zeros((maxM+1, maxM+1))

for M in range(1, maxM+1):
    Xtrain = np.append(Xtrain, (X1**M).reshape(-1, 1), axis=1)
    Xtest = np.append(Xtest, (X2**M).reshape(-1, 1), axis=1)
    Y1pred = reg.fit(Xtrain, Y1).predict(Xtrain)
    Y2pred = reg.predict(Xtest)
    
    coeff[M, 0] = reg.intercept_
    coeff[M, 1:M+1] = reg.coef_
    MSE1[M] = mean_squared_error(Y1, Y1pred)
    MSE2[M] = mean_squared_error(Y2, Y2pred)
    anorm2[M] = np.sum( reg.coef_**2 ) + reg.intercept_**2

fig1 = plt.figure(figsize=(16, 6))
ax11 = fig1.add_subplot(121)
ax11.plot(np.arange(1, maxM+1, 1), MSE1[1:], 'o-', linewidth=3, c='tab:blue', label='training MSE')
ax11.plot(np.arange(1, maxM+1, 1), MSE2[1:], 'o-', linewidth=3, c='tab:orange', label='test MSE')
ax11.tick_params(labelsize=14)
ax11.set_xlabel("number of regression variables", fontsize=18 )
ax11.legend(fontsize=18)

ax12 = fig1.add_subplot(122)
ax12.set_yscale('log')
ax12.plot(np.arange(1, maxM+1, 1), anorm2[1:], 'o-', linewidth=3, c='tab:blue', label='$||a||^2$')
ax12.tick_params(labelsize=14)
ax12.set_xlabel("number of regression variables", fontsize=18 )
ax12.legend(fontsize=18)


In [ ]:
# 多項式回帰関数のプロット
def plot_XY_and_PolyReg(M=1):
    XX = np.arange(np.min(X1), np.max(X1), 0.01)
    YY = coeff[M, 0]
    for i in range(1, M+1):
        YY += coeff[M, i] * XX**i
    
    fig = plt.figure(figsize=(16, 6))
    
    ax1 = fig.add_subplot(121)
    ax1.set_xlabel("X", size=20)
    ax1.set_ylabel("Y", size=20)
    ax1.set_xticks
    ax1.set_ylim(np.minimum(np.min(Y1), np.min(Y2)), np.maximum(np.max(Y1), np.max(Y2)))
    ax1.plot(XX, YY, linewidth=3)
    ax1.scatter(X1, Y1, s=100)
    ax1.tick_params(labelsize=12)
    ax1.text(-2, 7, 'MSE = %f'%(MSE1[M]), fontsize=20)
    
    ax2 = fig.add_subplot(122)
    ax2.set_xlabel("X", size=20)
    ax2.set_ylabel("Y", size=20)
    ax2.set_xticks
    ax2.set_ylim(np.minimum(np.min(Y1), np.min(Y2)), np.maximum(np.max(Y1), np.max(Y2)))
    ax2.plot(XX, YY, linewidth=3)
    ax2.scatter(X2, Y2, c='tab:orange', s=100)
    ax2.tick_params(labelsize=12)
    ax2.text(-2, 7, 'MSE = %f'%(MSE2[M]), fontsize=20)

In [ ]:
interact(plot_XY_and_PolyReg, M=(1, maxM))

In [ ]:
# 正則化パラメータ
alpha = 0.1

# フィッティングに用いる多項式の最大次数を１から1つずつ増やしていく
maxM = 20

# dataset
Xtrain = np.zeros((X1.size, 0))
Xtest = np.zeros((X2.size, 0))

# 回帰モデルの設定
ridge = linear_model.Ridge(alpha=alpha)

MSE1 = np.zeros(maxM+1)
MSE2 = np.zeros(maxM+1)
anorm2 = np.zeros(maxM+1)
coeff = np.zeros((maxM+1, maxM+1))

for M in range(1, maxM+1):
    Xtrain = np.append(Xtrain, (X1**M).reshape(-1, 1), axis=1)
    Xtest = np.append(Xtest, (X2**M).reshape(-1, 1), axis=1)
    Y1pred = ridge.fit(Xtrain, Y1).predict(Xtrain)
    Y2pred = ridge.predict(Xtest)
    
    coeff[M, 0] = ridge.intercept_
    coeff[M, 1:M+1] = ridge.coef_
    MSE1[M] = mean_squared_error(Y1, Y1pred)
    MSE2[M] = mean_squared_error(Y2, Y2pred)
    anorm2[M] = np.sum( ridge.coef_**2 ) + ridge.intercept_**2

fig1 = plt.figure(figsize=(16, 6))
ax11 = fig1.add_subplot(121)
ax11.plot(np.arange(1, maxM+1, 1), MSE1[1:], 'o-', linewidth=3, c='tab:blue', label='training MSE')
ax11.plot(np.arange(1, maxM+1, 1), MSE2[1:], 'o-', linewidth=3, c='tab:orange', label='test MSE')
ax11.tick_params(labelsize=14)
ax11.set_xlabel("number of regression variables", fontsize=18 )
ax11.legend(fontsize=18)

ax12 = fig1.add_subplot(122)
ax12.set_yscale('log')
ax12.plot(np.arange(1, maxM+1, 1), anorm2[1:], 'o-', linewidth=3, c='tab:blue', label='$||a||^2$')
ax12.tick_params(labelsize=14)
ax12.set_xlabel("number of regression variables", fontsize=18 )
ax12.legend(fontsize=18)


In [ ]:
# 多項式回帰関数のプロット
def plot_XY_and_PolyRegRidge(M=1):
    XX = np.arange(np.min(X1), np.max(X1), 0.01)
    YY = coeff[M, 0]
    for i in range(1, M+1):
        YY += coeff[M, i] * XX**i
    
    fig = plt.figure(figsize=(16, 6))
    
    ax1 = fig.add_subplot(121)
    ax1.set_xlabel("X", size=20)
    ax1.set_ylabel("Y", size=20)
    ax1.set_xticks
    ax1.set_ylim(np.minimum(np.min(Y1), np.min(Y2)), np.maximum(np.max(Y1), np.max(Y2)))
    ax1.plot(XX, YY, linewidth=3)
    ax1.scatter(X1, Y1, s=100)
    ax1.tick_params(labelsize=12)
    ax1.text(-2, 7, 'MSE = %f'%(MSE1[M]), fontsize=20)
    ax1.text(-2, 5, 'AIC = %f'%(-2*np.log(MSE1[M])+2*M), fontsize=20)    
    
    ax2 = fig.add_subplot(122)
    ax2.set_xlabel("X", size=20)
    ax2.set_ylabel("Y", size=20)
    ax2.set_xticks
    ax2.set_ylim(np.minimum(np.min(Y1), np.min(Y2)), np.maximum(np.max(Y1), np.max(Y2)))
    ax2.plot(XX, YY, linewidth=3)
    ax2.scatter(X2, Y2, c='tab:orange', s=100)
    ax2.tick_params(labelsize=12)
    ax2.text(-2, 7, 'MSE = %f'%(MSE2[M]), fontsize=20)
    ax2.text(-2, 5, 'AIC = %f'%(-2*np.log(MSE2[M])+2*M), fontsize=20)

In [ ]:
interact(plot_XY_and_PolyRegRidge, M=(1, maxM))

## 演習

Iris データセットに対して，Ridge 正則化によるフィッティングと AIC によるモデル選択（正則化は行わない）ではどのような差が起きるか． 
データを解析して両者を比較し，考察せよ．

<h3><div style="text-align: right;">以上</div></h3>